In [0]:
%python
%pip install databricks-labs-dqx --quiet
dbutils.library.restartPython()


In [0]:
%python
from databricks.labs.dqx.profiler.profiler import DQProfiler
from databricks.labs.dqx.profiler.generator import DQGenerator
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.config import WorkspaceFileChecksStorageConfig
from databricks.sdk import WorkspaceClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, countDistinct, when, isnan, sum as _sum, avg, min as _min, max as _max, stddev
import json
import time
import pandas as pd

In [0]:
%python
# Initialize
spark = SparkSession.builder.getOrCreate()
ws = WorkspaceClient()
profiler = DQProfiler(ws)

In [0]:
%python
spark.sql("USE CATALOG `workspace`")
spark.sql("USE SCHEMA `imdb_data_analysis`")

In [0]:
%python
# 2. Configuration and File Paths
# Base path for IMDB data in Unity Catalog Volume
BASE_PATH = "/Volumes/workspace/imdb_data_analysis/datastore"

# Define all 7 IMDB file paths (TSV files inside folders)
file_paths = {
    "title_basics": f"{BASE_PATH}/title.basics/title.basics.tsv",
    "title_ratings": f"{BASE_PATH}/title.ratings/title.ratings.tsv",
    "name_basics": f"{BASE_PATH}/name.basics/name.basics.tsv",
    "title_principals": f"{BASE_PATH}/title.principals/title.principals.tsv",
    "title_crew": f"{BASE_PATH}/title.crew/title.crew.tsv",
    "title_episode": f"{BASE_PATH}/title.episode/title.episode.tsv",
    "title_akas": f"{BASE_PATH}/title.akas/title.akas.tsv"
}

# Reference data paths 
iso_language_path = f"{BASE_PATH}/ISO_Language_Name/ISO_Language_Name.csv"
iso_country_path = f"{BASE_PATH}/iso_countries/iso_countries.csv"

# Output path for quality checks
OUTPUT_CHECKS_PATH = "/Workspace/Shared/imdb_dqx/checks"

In [0]:
%python
#3. Load All Datasets
# Load title.basics
df_title_basics = spark.read \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("quote", "") \
    .option("escape", "") \
    .option("nullValue", "\\N") \
    .csv(file_paths["title_basics"])


# Load title.ratings
df_title_ratings = spark.read \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("quote", "") \
    .option("escape", "") \
    .option("nullValue", "\\N") \
    .csv(file_paths["title_ratings"])


# Load name.basics
df_name_basics = spark.read \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("quote", "") \
    .option("escape", "") \
    .option("nullValue", "\\N") \
    .csv(file_paths["name_basics"])


# Load title.principals
df_title_principals = spark.read \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("quote", "") \
    .option("escape", "") \
    .option("nullValue", "\\N") \
    .csv(file_paths["title_principals"])


# Load title.crew
df_title_crew = spark.read \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("quote", "") \
    .option("escape", "") \
    .option("nullValue", "\\N") \
    .csv(file_paths["title_crew"])


# Load title.episode
df_title_episode = spark.read \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("quote", "") \
    .option("escape", "") \
    .option("nullValue", "\\N") \
    .csv(file_paths["title_episode"])

# Load title.akas
df_title_akas = spark.read \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("quote", "") \
    .option("escape", "") \
    .option("nullValue", "\\N") \
    .csv(file_paths["title_akas"])


# Store in dictionary for easier iteration
datasets = {
    "title_basics": df_title_basics,
    "title_ratings": df_title_ratings,
    "name_basics": df_name_basics,
    "title_principals": df_title_principals,
    "title_crew": df_title_crew,
    "title_episode": df_title_episode,
    "title_akas": df_title_akas
}

In [0]:
%python
# 4. Initial Data Exploration

def explore_dataset(df, name):
    """Quick exploration of dataset structure"""
    print(f"\n{'='*80}")
    print(f"{name.upper()}")
    print(f"{'='*80}")
    print(f"Records: {df.count():,}")
    print(f"Columns: {len(df.columns)}")
    print(f"\nSchema:")
    df.printSchema()
    print(f"\nSample Data:")
    display(df.limit(5))
    
    # Null count summary
    null_counts = df.select([
        _sum(when(col(c).isNull() | (col(c) == ""), 1).otherwise(0)).alias(c)
        for c in df.columns
    ]).collect()[0].asDict()
    
    total_records = df.count()
    print(f"\nNull/Empty Analysis:")
    for col_name, null_count in null_counts.items():
        null_pct = (null_count / total_records) * 100
        if null_pct > 0:
            print(f"  {col_name}: {null_count:,} ({null_pct:.2f}%)")

# Explore each dataset
for name, df in datasets.items():
    explore_dataset(df, name)

In [0]:
%python
# 5. Profiling Configurations

profiling_configs = {
    "title_basics": {
        "description": "Core title master data - Movies, TV shows, episodes",
        "business_critical_columns": ["tconst", "titleType", "primaryTitle", "startYear"],
        "options": {
            "sample_fraction": None,
            "limit": None,
            "round": True,
            "max_in_count": 20,
            "distinct_ratio": 0.001,
            "max_null_ratio": 0.15,
            "remove_outliers": True,
            "outlier_columns": ["runtimeMinutes", "startYear"],
            "num_sigmas": 3,
            "trim_strings": True,
            "max_empty_ratio": 0.01
        },
        "columns": ["tconst", "titleType", "primaryTitle", "originalTitle", "isAdult", "startYear", "endYear", "runtimeMinutes", "genres"]
    },
    
    "title_ratings": {
        "description": "Rating and voting metrics for titles",
        "business_critical_columns": ["tconst", "averageRating", "numVotes"],
        "options": {
            "sample_fraction": None,
            "limit": None,
            "round": True,
            "max_in_count": 100,
            "distinct_ratio": 0.01,
            "max_null_ratio": 0.0,
            "remove_outliers": True,
            "outlier_columns": ["numVotes"],
            "num_sigmas": 3,
            "trim_strings": False,
            "max_empty_ratio": 0.0
        },
        "columns": ["tconst", "averageRating", "numVotes"]
    },
    
    "name_basics": {
        "description": "Person/talent master data",
        "business_critical_columns": ["nconst", "primaryName"],
        "options": {
            "sample_fraction": None,
            "limit": None,
            "round": True,
            "max_in_count": 50,
            "distinct_ratio": 0.001,
            "max_null_ratio": 0.25,
            "remove_outliers": True,
            "outlier_columns": ["birthYear", "deathYear"],
            "num_sigmas": 3,
            "trim_strings": True,
            "max_empty_ratio": 0.01
        },
        "columns": ["nconst", "primaryName", "birthYear", "deathYear", "primaryProfession", "knownForTitles"]
    },
    
    "title_principals": {
        "description": "Cast and crew role assignments",
        "business_critical_columns": ["tconst", "nconst", "category"],
        "options": {
            "sample_fraction": None,
            "limit": None,
            "round": True,
            "max_in_count": 15,
            "distinct_ratio": 0.0001,
            "max_null_ratio": 0.5,
            "remove_outliers": True,
            "outlier_columns": ["ordering"],
            "num_sigmas": 3,
            "trim_strings": True,
            "max_empty_ratio": 0.3
        },
        "columns": ["tconst", "ordering", "nconst", "category", "job", "characters"]
    },
    
    "title_crew": {
        "description": "Directors and writers",
        "business_critical_columns": ["tconst", "directors", "writers"],
        "options": {
            "sample_fraction": None,
            "limit": None,
            "round": True,
            "max_in_count": 20,
            "distinct_ratio": 0.01,
            "max_null_ratio": 0.4,
            "remove_outliers": False,
            "num_sigmas": 3,
            "trim_strings": True,
            "max_empty_ratio": 0.35
        },
        "columns": ["tconst", "directors", "writers"]
    },
    
    "title_episode": {
        "description": "TV episode structure and hierarchy",
        "business_critical_columns": ["tconst", "parentTconst", "seasonNumber", "episodeNumber"],
        "options": {
            "sample_fraction": None,
            "limit": None,
            "round": True,
            "max_in_count": 30,
            "distinct_ratio": 0.001,
            "max_null_ratio": 0.3,
            "remove_outliers": True,
            "outlier_columns": ["seasonNumber", "episodeNumber"],
            "num_sigmas": 3,
            "trim_strings": True,
            "max_empty_ratio": 0.25
        },
        "columns": ["tconst", "parentTconst", "seasonNumber", "episodeNumber"]
    },
    
    "title_akas": {
        "description": "Regional title variations and localizations",
        "business_critical_columns": ["titleId", "region", "language", "title"],
        "options": {
            "sample_fraction": None,
            "limit": None,
            "round": True,
            "max_in_count": 100,
            "distinct_ratio": 0.0001,
            "max_null_ratio": 0.5,
            "remove_outliers": False,
            "num_sigmas": 3,
            "trim_strings": True,
            "max_empty_ratio": 0.4
        },
        "columns": ["titleId", "ordering", "title", "region", "language", "types", "attributes", "isOriginalTitle"]
    }
}

print("✓ Profiling configurations defined for all 7 datasets")
for name, config in profiling_configs.items():
    print(f"  {name}: {len(config['columns'])} columns | {config['description']}")


In [0]:
%python
# 6. Execute Data Profiling

# Profile all datasets with DQX
all_profiles = {}
all_summary_stats = {}
profiling_times = {}
profiling_errors = {}

print("Starting comprehensive profiling...")
print("="*80)

for dataset_name, df in datasets.items():
    config = profiling_configs[dataset_name]
    
    print(f"\n🔍 Profiling: {dataset_name}")
    print(f"   Description: {config['description']}")
    print(f"   Columns: {', '.join(config['columns'][:5])}{'...' if len(config['columns']) > 5 else ''}")
    
    try:
        # Select only specified columns
        df_filtered = df.select(config['columns'])
        
        start_time = time.time()
        
        # Profile dataset
        summary_stats, profiles = profiler.profile(
            df=df_filtered,
            options=config['options']
        )
        
        elapsed_time = time.time() - start_time
        
        # Store results
        all_summary_stats[dataset_name] = summary_stats
        all_profiles[dataset_name] = profiles
        profiling_times[dataset_name] = elapsed_time
        
        print(f"   ✓ Success: {len(profiles)} quality rules generated in {elapsed_time:.2f}s")
        
    except Exception as e:
        profiling_errors[dataset_name] = str(e)
        print(f"   ✗ Error: {str(e)}")

print("\n" + "="*80)
print(f"  Total rules: {sum(len(p) for p in all_profiles.values())}")
print(f"  Errors: {len(profiling_errors)}")


In [0]:
%python
# 7. Profiling Results Summary

# Create comprehensive profiling summary
summary_data = []

for dataset_name in datasets.keys():
    if dataset_name in all_profiles:
        config = profiling_configs[dataset_name]
        summary_data.append({
            'Dataset': dataset_name,
            'Description': config['description'],
            'Total Records': f"{datasets[dataset_name].count():,}",
            'Columns Profiled': len(config['columns']),
            'Quality Rules': len(all_profiles[dataset_name]),
            'Critical Columns': len(config['business_critical_columns']),
            'Profiling Time (s)': f"{profiling_times[dataset_name]:.2f}",
            'Status': '✓ Success'
        })
    else:
        summary_data.append({
            'Dataset': dataset_name,
            'Description': profiling_configs[dataset_name]['description'],
            'Total Records': f"{datasets[dataset_name].count():,}",
            'Columns Profiled': 0,
            'Quality Rules': 0,
            'Critical Columns': 0,
            'Profiling Time (s)': '0.00',
            'Status': f'✗ Error: {profiling_errors.get(dataset_name, "Unknown")}'
        })

summary_df = pd.DataFrame(summary_data)
display(summary_df)



In [0]:
%python
# 8. Detailed Quality Rules by Dataset
# Display generated profiles for each dataset
for dataset_name, profiles in all_profiles.items():
    print(f"\n{'='*100}")
    print(f"📋 {dataset_name.upper()} - {len(profiles)} Quality Rules")
    print(f"{'='*100}")
    print(f"Description: {profiling_configs[dataset_name]['description']}")
    print(f"Business Critical: {', '.join(profiling_configs[dataset_name]['business_critical_columns'])}")
    print(f"\nGenerated Rules:")
    
    for i, profile in enumerate(profiles, 1):
        print(f"  {i}. {profile}")


In [0]:
%python
#9. Data Quality Insights & Findings

def analyze_data_quality(dataset_name, df, summary_stats, profiles):
    """Analyze data quality metrics and generate insights"""
    print(f"\n{'='*80}")
    print(f"🔍 DATA QUALITY ANALYSIS: {dataset_name.upper()}")
    print(f"{'='*80}")
    
    config = profiling_configs[dataset_name]
    total_records = df.count()
    
    # 1. Completeness Analysis
    print(f"\n1️⃣ COMPLETENESS ANALYSIS")
    print(f"   Total Records: {total_records:,}")
    
    for col_name in config['columns']:
        if col_name in summary_stats:
            stats = summary_stats[col_name]
            null_count = stats.get('null_count', 0)
            null_pct = (null_count / total_records) * 100 if total_records > 0 else 0
            
            status = "✓" if null_pct < 5 else "⚠️" if null_pct < 20 else "❌"
            print(f"   {status} {col_name}: {null_pct:.2f}% null ({null_count:,} records)")
    
    # 2. Uniqueness Analysis
    print(f"\n2️⃣ UNIQUENESS ANALYSIS")
    for col_name in config['business_critical_columns']:
        if col_name in summary_stats:
            stats = summary_stats[col_name]
            distinct_count = stats.get('distinct_count', 0)
            distinct_pct = (distinct_count / total_records) * 100 if total_records > 0 else 0
            
            # Check if key column
            is_key = distinct_pct > 95
            status = "🔑" if is_key else "📊"
            print(f"   {status} {col_name}: {distinct_count:,} distinct ({distinct_pct:.2f}%)")
    
    # 3. Quality Rules Summary
    print(f"\n3️⃣ QUALITY RULES GENERATED")
    print(f"   Total Rules: {len(profiles)}")
    
    # Categorize rules
    null_checks = [p for p in profiles if 'IS NOT NULL' in str(p) or 'null' in str(p).lower()]
    range_checks = [p for p in profiles if 'BETWEEN' in str(p) or '>=' in str(p) or '<=' in str(p)]
    pattern_checks = [p for p in profiles if 'RLIKE' in str(p) or 'IN' in str(p)]
    
    print(f"   - Null/Completeness checks: {len(null_checks)}")
    print(f"   - Range/Boundary checks: {len(range_checks)}")
    print(f"   - Pattern/Format checks: {len(pattern_checks)}")
    print(f"   - Other checks: {len(profiles) - len(null_checks) - len(range_checks) - len(pattern_checks)}")

# Analyze each dataset
for dataset_name in all_profiles.keys():
    analyze_data_quality(
        dataset_name,
        datasets[dataset_name],
        all_summary_stats[dataset_name],
        all_profiles[dataset_name]
    )


In [0]:
%python
# 10. Generate and Save Quality Checks
# Generate quality checks from profiles

generator = DQGenerator(ws)
dq_engine = DQEngine(ws)

all_checks = {}
checks_saved = {}

print("Generating and saving quality checks...")
print("="*80)

for dataset_name, profiles in all_profiles.items():
    try:
        print(f"\n📝 {dataset_name}")
        
        # Generate quality rules
        checks = generator.generate_dq_rules(profiles)
        all_checks[dataset_name] = checks
        
        # Save checks to file
        checks_file = f"{OUTPUT_CHECKS_PATH}/{dataset_name}_checks.yml"
        dq_engine.save_checks(
            checks,
            config=WorkspaceFileChecksStorageConfig(location=checks_file)
        )
        checks_saved[dataset_name] = checks_file
        
        print(f"   ✓ {len(checks)} checks saved to: {checks_file}")
        
    except Exception as e:
        print(f"   ✗ Error saving checks: {str(e)}")

print("\n" + "="*80)
print(f"✓ Quality checks generated and saved")
print(f"  Location: {OUTPUT_CHECKS_PATH}")
print(f"  Total files: {len(checks_saved)}")



In [0]:
%python
# MAGIC ## 11. ISO Reference Data Validation


print("🌍 ISO REFERENCE DATA VALIDATION")
print("="*80)

try:
    # Load ISO reference CSV files
    df_languages = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv(iso_language_path)
    print(f"✓ ISO Languages loaded: {df_languages.count():,} records")
    
    df_countries = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv(iso_country_path)
    print(f"✓ ISO Countries loaded: {df_countries.count():,} records")
    
    # Show sample data
    print("\nISO Languages Sample:")
    display(df_languages.limit(10))
    
    print("\nISO Countries Sample:")
    display(df_countries.limit(10))
    
    # Show column names for validation
    print(f"\nLanguage table columns: {df_languages.columns}")
    print(f"Country table columns: {df_countries.columns}")
    
    # Get distinct values from title_akas
    print(f"\n1️⃣ Language codes in title_akas")
    akas_languages = datasets['title_akas'].select('language').distinct().filter(col('language').isNotNull())
    total_distinct_langs = akas_languages.count()
    print(f"   Total distinct languages: {total_distinct_langs}")
    
    print(f"\n2️⃣ Region codes in title_akas")
    akas_regions = datasets['title_akas'].select('region').distinct().filter(col('region').isNotNull())
    total_distinct_regions = akas_regions.count()
    print(f"   Total distinct regions: {total_distinct_regions}")
    
    print(f"\n✓ ISO reference data validation complete")
    
except Exception as e:
    print(f"⚠️ ISO reference data validation skipped: {str(e)}")
    print("   Ensure ISO reference files exist in the specified paths")
    print(f"   Expected paths:")
    print(f"   - Languages: {iso_language_path}")
    print(f"   - Countries: {iso_country_path}")

In [0]:
%python
# 12. Multi-Value Field Analysis

from pyspark.sql.functions import split, explode, size, trim

print("🔢 MULTI-VALUE FIELD ANALYSIS")
print("="*80)

# 1. Genres in title_basics
print("\n1️⃣ Genre Analysis (title_basics.genres)")
genres_df = datasets['title_basics'].select(
    col('tconst'),
    split(col('genres'), ',').alias('genre_array')
).withColumn('genre_count', size(col('genre_array')))

genres_with_data = genres_df.filter(col('genre_count') > 0)
print(f"   Records with genres: {genres_with_data.count():,}")
print(f"   Average genres per title: {genres_df.agg(avg('genre_count')).collect()[0][0]:.2f}")
print(f"   Max genres per title: {genres_df.agg(_max('genre_count')).collect()[0][0]}")

# Top genres
top_genres = genres_df.select(explode(col('genre_array')).alias('genre')) \
    .filter(col('genre').isNotNull() & (col('genre') != '')) \
    .groupBy('genre') \
    .count() \
    .orderBy(col('count').desc()) \
    .limit(15)
print(f"\n   Top 15 Genres:")
display(top_genres)

# 2. Primary Professions in name_basics
print("\n2️⃣ Profession Analysis (name_basics.primaryProfession)")
professions_df = datasets['name_basics'].select(
    col('nconst'),
    split(col('primaryProfession'), ',').alias('profession_array')
).withColumn('profession_count', size(col('profession_array')))

professions_with_data = professions_df.filter(col('profession_count') > 0)
print(f"   People with professions: {professions_with_data.count():,}")
print(f"   Average professions per person: {professions_df.agg(avg('profession_count')).collect()[0][0]:.2f}")
print(f"   Max professions per person: {professions_df.agg(_max('profession_count')).collect()[0][0]}")

# Top professions
top_professions = professions_df.select(explode(col('profession_array')).alias('profession')) \
    .filter(col('profession').isNotNull() & (col('profession') != '')) \
    .groupBy('profession') \
    .count() \
    .orderBy(col('count').desc()) \
    .limit(15)
print(f"\n   Top 15 Professions:")
display(top_professions)

# 3. Directors in title_crew
print("\n3️⃣ Directors Analysis (title_crew.directors)")
directors_df = datasets['title_crew'].filter(col('directors').isNotNull()) \
    .select(
        col('tconst'),
        split(col('directors'), ',').alias('director_array')
    ).withColumn('director_count', size(col('director_array')))

print(f"   Titles with directors: {directors_df.count():,}")
print(f"   Average directors per title: {directors_df.agg(avg('director_count')).collect()[0][0]:.2f}")
print(f"   Max directors per title: {directors_df.agg(_max('director_count')).collect()[0][0]}")

# 4. Writers in title_crew
print("\n4️⃣ Writers Analysis (title_crew.writers)")
writers_df = datasets['title_crew'].filter(col('writers').isNotNull()) \
    .select(
        col('tconst'),
        split(col('writers'), ',').alias('writer_array')
    ).withColumn('writer_count', size(col('writer_array')))

print(f"   Titles with writers: {writers_df.count():,}")
print(f"   Average writers per title: {writers_df.agg(avg('writer_count')).collect()[0][0]:.2f}")
print(f"   Max writers per title: {writers_df.agg(_max('writer_count')).collect()[0][0]}")

print(f"\n{'='*80}")
print(f"✓ Multi-value field analysis complete")
print(f"\n💡 Insights:")
print(f"   - Genres: Useful for BRIDGE_Title_Genre table design")
print(f"   - Professions: Useful for BRIDGE_Person_Profession table design")
print(f"   - Directors/Writers: Will be normalized in FACT_Title_Person_Role")